## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-11-29 19:32:12.489455: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)
application_df.head(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

def createOtherColumn(df, col, cutoff, rep_str="Other"):
    vc = df[col].value_counts()
    to_replace = list()
    for ind in list(vc.index):
        if vc[ind] < cutoff:
            to_replace.append(ind)

    
    for r in to_replace:
        df[col] = df[col].replace(r, rep_str)

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_type_vc = application_df["APPLICATION_TYPE"].value_counts()
print(app_type_vc)
createOtherColumn(application_df, 'APPLICATION_TYPE', 200)
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64
APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [11]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 200
application_types_to_replace = list()
for ind in list(app_type_vc.index):
    if app_type_vc[ind] < cutoff:
        application_types_to_replace.append(ind)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
class_vc = application_df["CLASSIFICATION"].value_counts()
print(class_vc[class_vc > 1])
createOtherColumn(application_df, 'CLASSIFICATION', 1000)
print(application_df['CLASSIFICATION'].value_counts())




CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [16]:
ask_amt_vc = application_df["ASK_AMT"].value_counts()
value_dict = dict()
for ind in ask_amt_vc.index:
    cur_val = ask_amt_vc[ind]
    if cur_val not in value_dict:
        value_dict[cur_val] = 1
    else:
        value_dict[cur_val] += 1
print(value_dict)
createOtherColumn(application_df, "ASK_AMT", 10, "GT_5000")
print(application_df["ASK_AMT"].value_counts())
      
ask_amt_vc = application_df["ASK_AMT"].value_counts()
for ind in ask_amt_vc.index:
    print(ind)
    print(application_df[application_df["ASK_AMT"] == ind]["IS_SUCCESSFUL"].value_counts())

income_vc = application_df["INCOME_AMT"].value_counts()
print(income_vc)
for ind in income_vc.index:
    print(ind)
    print(application_df[application_df["INCOME_AMT"] == ind]["IS_SUCCESSFUL"].value_counts())

use_case_vc = application_df["USE_CASE"].value_counts()
print(use_case_vc)
for ind in use_case_vc.index:
    print(ind)
    print(application_df[application_df["USE_CASE"] == ind]["IS_SUCCESSFUL"].value_counts())
    
organization_vc = application_df["ORGANIZATION"].value_counts()
print(organization_vc)
for ind in organization_vc.index:
    print(ind)
    print(application_df[application_df["ORGANIZATION"] == ind]["IS_SUCCESSFUL"].value_counts())
    
status_vc = application_df["STATUS"].value_counts()
print(status_vc)
for ind in status_vc.index:
    print(ind)
    print(application_df[application_df["STATUS"] == ind]["IS_SUCCESSFUL"].value_counts())    

{25398: 1, 8901: 1}
ASK_AMT
5000       25398
GT_5000     8901
Name: count, dtype: int64
5000
IS_SUCCESSFUL
1    13303
0    12095
Name: count, dtype: int64
GT_5000
IS_SUCCESSFUL
1    4958
0    3943
Name: count, dtype: int64
INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64
0
IS_SUCCESSFUL
1    12577
0    11811
Name: count, dtype: int64
25000-99999
IS_SUCCESSFUL
1    2135
0    1612
Name: count, dtype: int64
100000-499999
IS_SUCCESSFUL
1    1952
0    1422
Name: count, dtype: int64
1M-5M
IS_SUCCESSFUL
0    510
1    445
Name: count, dtype: int64
1-9999
IS_SUCCESSFUL
1    553
0    175
Name: count, dtype: int64
10000-24999
IS_SUCCESSFUL
1    368
0    175
Name: count, dtype: int64
10M-50M
IS_SUCCESSFUL
0    143
1     97
Name: count, dtype: int64
5M-10M
IS_SUCCESSFUL
0    98
1    87
Name: count, dtype: i

In [14]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_vc[class_vc > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [17]:
application_df.drop(["STATUS"], axis=1, inplace=True)
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,GT_5000,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,GT_5000,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,GT_5000,1


In [18]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1000
classifications_to_replace = list()
for ind in list(class_vc.index):
    if class_vc[ind] < cutoff:
        classifications_to_replace.append(ind)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
ull_df = pd.get_dummies(application_df)
full_df

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_GT_5000
0,1,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
1,1,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
2,0,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,True,False
3,1,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,1,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
34295,0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
34296,0,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
34297,1,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [25]:
# Split our preprocessed data into our features and target arrays

X = full_df.drop(["IS_SUCCESSFUL"], axis=1)
y = full_df[["IS_SUCCESSFUL"]]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Users/payal/anaconda3/envs/module19/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/payal/anaconda3/envs/module19/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/payal/anaconda3/envs/module19/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


## Compile, Train and Evaluate the Model

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
initializer = tf.keras.initializers.Ones()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=len(X.columns), kernel_initializer=initializer))

nn.add(tf.keras.layers.Dense(units=110, activation="sigmoid", kernel_initializer=initializer))

nn.add(tf.keras.layers.Dense(units=50, activation="relu", kernel_initializer=initializer))

nn.add(tf.keras.layers.Dense(units=20, activation="relu", kernel_initializer=initializer))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu", kernel_initializer=initializer))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 110)               8910      
                                                                 
 dense_2 (Dense)             (None, 50)                5550      
                                                                 
 dense_3 (Dense)             (None, 20)                1020      
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 19,021
Trainable params: 19,021
Non-trainable params: 0
_________________________________________________________________


2023-11-29 19:49:37.217337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
nn_fit = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 888us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 2/150
804/804 [==============================] - 1s 877us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 3/150
804/804 [==============================] - 1s 873us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 4/150
804/804 [==============================] - 1s 886us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 5/150
804/804 [==============================] - 1s 868us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 6/150
804/804 [==============================] - 1s 922us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 7/150
804/804 [==============================] - 1s 908us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 7.1432 - accuracy: 0.5316
Epoch 9/150
804/804 [==============================] - 1s 856us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 10/150
804/804 [==============================] - 1s 801us/step - los

Epoch 80/150
804/804 [==============================] - 1s 818us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 81/150
804/804 [==============================] - 1s 814us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 82/150
804/804 [==============================] - 1s 903us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 83/150
804/804 [==============================] - 1s 857us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 84/150
804/804 [==============================] - 1s 875us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 85/150
804/804 [==============================] - 1s 831us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 86/150
804/804 [==============================] - 1s 814us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 87/150
804/804 [==============================] - 1s 970us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 88/150
804/804 [==============================] - 1s 910us/step - loss: 7.1432 - accuracy: 0.5316
Epoch 89/150
804/804 [==============================] - 1s 809us

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 7.0920 - accuracy: 0.5349 - 171ms/epoch - 638us/step
Loss: 7.0920023918151855, Accuracy: 0.5349271297454834


In [33]:
# Export our model to HDF5 file
nn.save(filepath="AlphabetSoupCharity.h5", save_format="h5")